In [1]:
using POMDPs # for MDP type
using DiscreteValueIteration
using POMDPPolicies
using POMDPModelTools #for sparse cat 
using Parameters
using Random
using Plots; default(fontfamily="Computer Modern", framestyle=:box) # LaTex-style
using QuickPOMDPs
using Distributions 

In [2]:
Random.seed!(123)

MersenneTwister(123)

**States**

In [3]:
# A state s in the evacuation problem is a discrete set of 4 values: 
# chairs remaining, time remaining, family size, and visa status
struct State
    c::Int # chairs remaining 
    t::Int # time remaining 
    f::Int # family size 
    v::Int # visa status 
end 

**Environment Parameters**

In [4]:
# assume families normally distributed as integers around average family size in afghanistan of 8. 
# Truncated to include 1 or greater and rounded to be integers 
@with_kw struct EvacuationParameters
    # need the probability that a particular family shows up at the door....
    family_sizes::Vector{Int} =  [trunc(Int, x) for x in rand(TruncatedNormal(3, 1, 1, Inf), 10)] 
    visa_status::Vector{Int} = [-4, -3, -2, 0, 1, 2, 3, 4] #TODO: map to various status 
    capacity::Int = 20
    time::Int = 60
    size::Tuple{Int, Int} = (length(visa_status), length(family_sizes)) # size of grid 
    p_transition::Real = 0.8 # don't we always transition into this since time moves forward? I'm confused... 
    null_state::State = State(0, 0, 0 ,0) # is there someway to do this???
end


EvacuationParameters

In [5]:
params = EvacuationParameters(); 

In [6]:
# The state space S for the evacuation problem is the set of 
# all combinations 
# Question: Do I need the end state here? 
#𝒮 = [[State(c, t, f, v) for  c=1:params.capacity, t=1:params.time, f=1:params.size[2], v=1:params.size[1]]]
𝒮 = []

for c in 0:params.capacity # capacity ends at 0 
    for t in 0:params.time # time ends at 0 
        for f in params.family_sizes # family size here we should have the ACTUAL family sizes 
            for v in params.visa_status # actual visa statuses 
                new = State(c, t, f, v) 
                𝒮 = [𝒮; new]
            end
        end        
    end
end
𝒮 = [𝒮; params.null_state]

# are we s


102481-element Vector{Any}:
 State(0, 0, 4, -4)
 State(0, 0, 4, -3)
 State(0, 0, 4, -2)
 State(0, 0, 4, 0)
 State(0, 0, 4, 1)
 State(0, 0, 4, 2)
 State(0, 0, 4, 3)
 State(0, 0, 4, 4)
 State(0, 0, 5, -4)
 State(0, 0, 5, -3)
 State(0, 0, 5, -2)
 State(0, 0, 5, 0)
 State(0, 0, 5, 1)
 ⋮
 State(20, 60, 3, 2)
 State(20, 60, 3, 3)
 State(20, 60, 3, 4)
 State(20, 60, 2, -4)
 State(20, 60, 2, -3)
 State(20, 60, 2, -2)
 State(20, 60, 2, 0)
 State(20, 60, 2, 1)
 State(20, 60, 2, 2)
 State(20, 60, 2, 3)
 State(20, 60, 2, 4)
 State(0, 0, 0, 0)

In [7]:
@show 𝒮

𝒮 = Any[State(0, 0, 4, -4), State(0, 0, 4, -3), State(0, 0, 4, -2), State(0, 0, 4, 0), State(0, 0, 4, 1), State(0, 0, 4, 2), State(0, 0, 4, 3), State(0, 0, 4, 4), State(0, 0, 5, -4), State(0, 0, 5, -3), State(0, 0, 5, -2), State(0, 0, 5, 0), State(0, 0, 5, 1), State(0, 0, 5, 2), State(0, 0, 5, 3), State(0, 0, 5, 4), State(0, 0, 4, -4), State(0, 0, 4, -3), State(0, 0, 4, -2), State(0, 0, 4, 0), State(0, 0, 4, 1), State(0, 0, 4, 2), State(0, 0, 4, 3), State(0, 0, 4, 4), State(0, 0, 3, -4), State(0, 0, 3, -3), State(0, 0, 3, -2), State(0, 0, 3, 0), State(0, 0, 3, 1), State(0, 0, 3, 2), State(0, 0, 3, 3), State(0, 0, 3, 4), State(0, 0, 2, -4), State(0, 0, 2, -3), State(0, 0, 2, -2), State(0, 0, 2, 0), State(0, 0, 2, 1), State(0, 0, 2, 2), State(0, 0, 2, 3), State(0, 0, 2, 4), State(0, 0, 2, -4), State(0, 0, 2, -3), State(0, 0, 2, -2), State(0, 0, 2, 0), State(0, 0, 2, 1), State(0, 0, 2, 2), State(0, 0, 2, 3), State(0, 0, 2, 4), State(0, 0, 3, -4), State(0, 0, 3, -3), State(0, 0, 3, -2), St

, 3, 4), State(0, 40, 2, -4), State(0, 40, 2, -3), State(0, 40, 2, -2), State(0, 40, 2, 0), State(0, 40, 2, 1), State(0, 40, 2, 2), State(0, 40, 2, 3), State(0, 40, 2, 4), State(0, 40, 3, -4), State(0, 40, 3, -3), State(0, 40, 3, -2), State(0, 40, 3, 0), State(0, 40, 3, 1), State(0, 40, 3, 2), State(0, 40, 3, 3), State(0, 40, 3, 4), State(0, 40, 2, -4), State(0, 40, 2, -3), State(0, 40, 2, -2), State(0, 40, 2, 0), State(0, 40, 2, 1), State(0, 40, 2, 2), State(0, 40, 2, 3), State(0, 40, 2, 4), State(0, 41, 4, -4), State(0, 41, 4, -3), State(0, 41, 4, -2), State(0, 41, 4, 0), State(0, 41, 4, 1), State(0, 41, 4, 2), State(0, 41, 4, 3), State(0, 41, 4, 4), State(0, 41, 5, -4), State(0, 41, 5, -3), State(0, 41, 5, -2), State(0, 41, 5, 0), State(0, 41, 5, 1), State(0, 41, 5, 2), State(0, 41, 5, 3), State(0, 41, 5, 4), State(0, 41, 4, -4), State(0, 41, 4, -3), State(0, 41, 4, -2), State(0, 41, 4, 0), State(0, 41, 4, 1), State(0, 41, 4, 2), State(0, 41, 4, 3), State(0, 41, 4, 4), State(0, 41, 

1, 20, 3, 4), State(1, 20, 2, -4), State(1, 20, 2, -3), State(1, 20, 2, -2), State(1, 20, 2, 0), State(1, 20, 2, 1), State(1, 20, 2, 2), State(1, 20, 2, 3), State(1, 20, 2, 4), State(1, 20, 2, -4), State(1, 20, 2, -3), State(1, 20, 2, -2), State(1, 20, 2, 0), State(1, 20, 2, 1), State(1, 20, 2, 2), State(1, 20, 2, 3), State(1, 20, 2, 4), State(1, 20, 3, -4), State(1, 20, 3, -3), State(1, 20, 3, -2), State(1, 20, 3, 0), State(1, 20, 3, 1), State(1, 20, 3, 2), State(1, 20, 3, 3), State(1, 20, 3, 4), State(1, 20, 2, -4), State(1, 20, 2, -3), State(1, 20, 2, -2), State(1, 20, 2, 0), State(1, 20, 2, 1), State(1, 20, 2, 2), State(1, 20, 2, 3), State(1, 20, 2, 4), State(1, 20, 3, -4), State(1, 20, 3, -3), State(1, 20, 3, -2), State(1, 20, 3, 0), State(1, 20, 3, 1), State(1, 20, 3, 2), State(1, 20, 3, 3), State(1, 20, 3, 4), State(1, 20, 2, -4), State(1, 20, 2, -3), State(1, 20, 2, -2), State(1, 20, 2, 0), State(1, 20, 2, 1), State(1, 20, 2, 2), State(1, 20, 2, 3), State(1, 20, 2, 4), State(1,

4), State(1, 60, 3, -4), State(1, 60, 3, -3), State(1, 60, 3, -2), State(1, 60, 3, 0), State(1, 60, 3, 1), State(1, 60, 3, 2), State(1, 60, 3, 3), State(1, 60, 3, 4), State(1, 60, 2, -4), State(1, 60, 2, -3), State(1, 60, 2, -2), State(1, 60, 2, 0), State(1, 60, 2, 1), State(1, 60, 2, 2), State(1, 60, 2, 3), State(1, 60, 2, 4), State(1, 60, 3, -4), State(1, 60, 3, -3), State(1, 60, 3, -2), State(1, 60, 3, 0), State(1, 60, 3, 1), State(1, 60, 3, 2), State(1, 60, 3, 3), State(1, 60, 3, 4), State(1, 60, 2, -4), State(1, 60, 2, -3), State(1, 60, 2, -2), State(1, 60, 2, 0), State(1, 60, 2, 1), State(1, 60, 2, 2), State(1, 60, 2, 3), State(1, 60, 2, 4), State(2, 0, 4, -4), State(2, 0, 4, -3), State(2, 0, 4, -2), State(2, 0, 4, 0), State(2, 0, 4, 1), State(2, 0, 4, 2), State(2, 0, 4, 3), State(2, 0, 4, 4), State(2, 0, 5, -4), State(2, 0, 5, -3), State(2, 0, 5, -2), State(2, 0, 5, 0), State(2, 0, 5, 1), State(2, 0, 5, 2), State(2, 0, 5, 3), State(2, 0, 5, 4), State(2, 0, 4, -4), State(2, 0, 4,

, 4, 4), State(2, 40, 3, -4), State(2, 40, 3, -3), State(2, 40, 3, -2), State(2, 40, 3, 0), State(2, 40, 3, 1), State(2, 40, 3, 2), State(2, 40, 3, 3), State(2, 40, 3, 4), State(2, 40, 2, -4), State(2, 40, 2, -3), State(2, 40, 2, -2), State(2, 40, 2, 0), State(2, 40, 2, 1), State(2, 40, 2, 2), State(2, 40, 2, 3), State(2, 40, 2, 4), State(2, 40, 2, -4), State(2, 40, 2, -3), State(2, 40, 2, -2), State(2, 40, 2, 0), State(2, 40, 2, 1), State(2, 40, 2, 2), State(2, 40, 2, 3), State(2, 40, 2, 4), State(2, 40, 3, -4), State(2, 40, 3, -3), State(2, 40, 3, -2), State(2, 40, 3, 0), State(2, 40, 3, 1), State(2, 40, 3, 2), State(2, 40, 3, 3), State(2, 40, 3, 4), State(2, 40, 2, -4), State(2, 40, 2, -3), State(2, 40, 2, -2), State(2, 40, 2, 0), State(2, 40, 2, 1), State(2, 40, 2, 2), State(2, 40, 2, 3), State(2, 40, 2, 4), State(2, 40, 3, -4), State(2, 40, 3, -3), State(2, 40, 3, -2), State(2, 40, 3, 0), State(2, 40, 3, 1), State(2, 40, 3, 2), State(2, 40, 3, 3), State(2, 40, 3, 4), State(2, 40, 

3, 19, 2, 4), State(3, 20, 4, -4), State(3, 20, 4, -3), State(3, 20, 4, -2), State(3, 20, 4, 0), State(3, 20, 4, 1), State(3, 20, 4, 2), State(3, 20, 4, 3), State(3, 20, 4, 4), State(3, 20, 5, -4), State(3, 20, 5, -3), State(3, 20, 5, -2), State(3, 20, 5, 0), State(3, 20, 5, 1), State(3, 20, 5, 2), State(3, 20, 5, 3), State(3, 20, 5, 4), State(3, 20, 4, -4), State(3, 20, 4, -3), State(3, 20, 4, -2), State(3, 20, 4, 0), State(3, 20, 4, 1), State(3, 20, 4, 2), State(3, 20, 4, 3), State(3, 20, 4, 4), State(3, 20, 3, -4), State(3, 20, 3, -3), State(3, 20, 3, -2), State(3, 20, 3, 0), State(3, 20, 3, 1), State(3, 20, 3, 2), State(3, 20, 3, 3), State(3, 20, 3, 4), State(3, 20, 2, -4), State(3, 20, 2, -3), State(3, 20, 2, -2), State(3, 20, 2, 0), State(3, 20, 2, 1), State(3, 20, 2, 2), State(3, 20, 2, 3), State(3, 20, 2, 4), State(3, 20, 2, -4), State(3, 20, 2, -3), State(3, 20, 2, -2), State(3, 20, 2, 0), State(3, 20, 2, 1), State(3, 20, 2, 2), State(3, 20, 2, 3), State(3, 20, 2, 4), State(3,

4), State(3, 60, 4, -4), State(3, 60, 4, -3), State(3, 60, 4, -2), State(3, 60, 4, 0), State(3, 60, 4, 1), State(3, 60, 4, 2), State(3, 60, 4, 3), State(3, 60, 4, 4), State(3, 60, 3, -4), State(3, 60, 3, -3), State(3, 60, 3, -2), State(3, 60, 3, 0), State(3, 60, 3, 1), State(3, 60, 3, 2), State(3, 60, 3, 3), State(3, 60, 3, 4), State(3, 60, 2, -4), State(3, 60, 2, -3), State(3, 60, 2, -2), State(3, 60, 2, 0), State(3, 60, 2, 1), State(3, 60, 2, 2), State(3, 60, 2, 3), State(3, 60, 2, 4), State(3, 60, 2, -4), State(3, 60, 2, -3), State(3, 60, 2, -2), State(3, 60, 2, 0), State(3, 60, 2, 1), State(3, 60, 2, 2), State(3, 60, 2, 3), State(3, 60, 2, 4), State(3, 60, 3, -4), State(3, 60, 3, -3), State(3, 60, 3, -2), State(3, 60, 3, 0), State(3, 60, 3, 1), State(3, 60, 3, 2), State(3, 60, 3, 3), State(3, 60, 3, 4), State(3, 60, 2, -4), State(3, 60, 2, -3), State(3, 60, 2, -2), State(3, 60, 2, 0), State(3, 60, 2, 1), State(3, 60, 2, 2), State(3, 60, 2, 3), State(3, 60, 2, 4), State(3, 60, 3, -4

Excessive output truncated after 524295 bytes.

102481-element Vector{Any}:
 State(0, 0, 4, -4)
 State(0, 0, 4, -3)
 State(0, 0, 4, -2)
 State(0, 0, 4, 0)
 State(0, 0, 4, 1)
 State(0, 0, 4, 2)
 State(0, 0, 4, 3)
 State(0, 0, 4, 4)
 State(0, 0, 5, -4)
 State(0, 0, 5, -3)
 State(0, 0, 5, -2)
 State(0, 0, 5, 0)
 State(0, 0, 5, 1)
 ⋮
 State(20, 60, 3, 2)
 State(20, 60, 3, 3)
 State(20, 60, 3, 4)
 State(20, 60, 2, -4)
 State(20, 60, 2, -3)
 State(20, 60, 2, -2)
 State(20, 60, 2, 0)
 State(20, 60, 2, 1)
 State(20, 60, 2, 2)
 State(20, 60, 2, 3)
 State(20, 60, 2, 4)
 State(0, 0, 0, 0)

In [8]:
@show size(𝒮)
#@show size[1,1,1,1]
#(20, 60, 10, 10)
#length(𝒮[1]) = 120000 = 100*60*10*10

size(𝒮) = (102481,)


(102481,)

**Actions**

In [9]:
# the possible actions are whether accept or reject a family at the gate 
@enum Action REJECT ACCEPT

In [10]:
𝒜 = [REJECT, ACCEPT]

2-element Vector{Action}:
 REJECT::Action = 0
 ACCEPT::Action = 1

In [11]:
length(𝒜)

2

In [37]:
# create policy grid showing the best action in each state at a particular time and capacity 
#State(c, t, f, v)
function policy_grid(policy::Policy, xmax::Int, ymax::Int, time::Int, capacity::Int)
    viz_action = Dict(REJECT => "",
                  ACCEPT => "+",)
    grid = Array{String}(undef, xmax, ymax)
    for x = 1:xmax, y = 1:xmax
        s = State(capacity, time, x, y) # x is family size, v is visa status 
        grid[x,y] = viz_action[action(policy, s)] # need to define this action space 
    end
    return grid
end


policy_grid (generic function with 1 method)

In [12]:
v′ = rand(params.visa_status, 1)

1-element Vector{Int64}:
 3

In [13]:
v′ = rand(params.visa_status, 1)[1]

1

In [14]:
# only inbounds if room for the family [assuming would not separate even though might]
# and if time is available to enter the airport 
validtime(s::State) = 0 ≤ s.t 

validtime (generic function with 1 method)

In [15]:
validcapacity(s::State) = 0 ≤ s.c 

validcapacity (generic function with 1 method)

**Transition Function** 

In [16]:
#XXXXXXXXXXXXXXXXXXXXXXXXXNote from Lilian:XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
# check out https://juliapomdp.github.io/POMDPModelTools.jl/stable/distributions/
# we are supposed to return a sparsecat structure with all the states and the prob of going to them next
# so i think this is incorrect because we seem to be not returning that
# SparseCat(values, probabilities) to create a sparse categorical distribution.
#textbook page 148

#Other notes:
#T = P(S' | S, A), so the probs are always 100% for each action 

#I figured it out, just havent coded it yet, just have to send back a SparseCat with 
# if action = accept, new state (S') to be time -1 and seats -1 100% and the rest 0%
# if action = reject, send back exact same state space (s') but with time - 1 100% of the time

#this is assuming an accept means 100% you get on the plane and reject means you can't sneak onto it. I 
#I think it might be realistic to make an accept give you a 80% chance of actually getting a seat if accepted.
#i read your article and it seemed like it was so disorganized that some of the accepted people didnt 
#make it. 

#need to add global rep of all the states so we can check it

# think about how to fix the transition function 
# The transition function returns a distribution over next states given the current state and an action .
function T(s::State, a::Action)
    # randomly sample a family size and visa status 
    if s.t == 0
        return Deterministic(params.null_state)
    end
        
    #null_state = params.nullstate
        
    f′ = rand(params.family_sizes, 1)[1]
    v′ = rand(params.visa_status, 1)[1] # TODO: possibly make this weighted in some way 
    #next_states = Vector{State}(undef, Nₐ) 
    if a == ACCEPT 
        Nₐ = 2 #80% get a seat, 20% lost it due to poor organization
        next_states = Vector{State}(undef, Nₐ) 
        probabilities = [.80, .20]
        next_state_accept = State(s.c - s.f, s.t - 1, f′, v′) # they get seats
        next_state_reject = State(s.c, s.t - 1, f′, v′)
        if validcapacity(next_state_accept) 
            next_states[1] = next_state_accept
        else
            probabilities = [0, 1] #no room for full family :( so we make probability 0 to accept and 1 reject
        end
        next_states[2] = next_state_reject  # no need to check capacity if rejecting 

    elseif a == REJECT
        Nₐ = 1
        next_states = Vector{State}(undef, Nₐ) 
        probabilities = [1.0]
        next_states[1] = State(s.c, s.t - 1, f′, v′) #where they don't get seats
    
        # handle out-of-bounds transitions
    else 
        Nₐ = 1
        next_states = Vector{State}(undef, Nₐ) 
        probabilities = [1.0]
        next_states[1] = s
    end
    return SparseCat(next_states, probabilities)
end

    

T (generic function with 1 method)

In [17]:
# struct State
#     c::Int # chairs remaining 
#     t::Int # time remaining 
# #     f::Int # family size 
# #     v::Int # visa status 
# # end 
T(State(6, 4, 4, 5), ACCEPT)


# ACCEPT: State(50, 9, ANYTHING, ANYTHING)
# ACCEPT: State(42, 9, ANYTHING, ANYTHING)
# REJECT: State(50, 9, ANYTHING, ANYTHING)

                               SparseCat distribution           
                     ┌                                        ┐ 
   State(2, 3, 5, 0) ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   State(6, 3, 5, 0) ┤■■■■■■■■■ 0.2                             
                     └                                        ┘ 

**Reward Function**

In [18]:
function R(s::State, a::Action)
    # reward is just the visa status times family size i think! 
    if a == ACCEPT
        return s.v*s.f
    end
    return 0
end 

R (generic function with 1 method)

In [19]:
#render(mdp; show_rewards=true)
#@bind γ Slider(0:0.05:1, default=0.95, show_value=true)
# hard code for now. maybe come back to it. 
γ = 0.95

0.95

In [20]:
# when time is 0...possibly encode this into the parameters 
#termination(s::State) = (s.t == 0 || s.c == 0)

termination(s::State) = (s.t == params.null_state.t && s.c == params.null_state.c)

termination (generic function with 1 method)

**MDP Formulation**

In [21]:
# We define the Airport abstract MDP type so we can reference it in other methods.
abstract type Evacuation <: MDP{State, Action} end

In [22]:
  # struct State
        #     c::Int # chairs remaining 
        #     t::Int # time remaining 
        #     f::Int # family size 
        #     v::Int # visa status 
        # end 
c_initial = params.capacity
t_initial = params.time
f_initial = rand(params.family_sizes, 1)[1]
v_initial = rand(params.visa_status, 1)[1]

initial_state = State(c_initial, t_initial, f_initial, v_initial)

State(20, 60, 3, 3)

In [23]:
statetype = typeof(initial_state)
@show initialstate_array = [initial_state]

initialstate_array = [initial_state] = State[State(20, 60, 3, 3)]


1-element Vector{State}:
 State(20, 60, 3, 3)

In [24]:
initialstate_array

1-element Vector{State}:
 State(20, 60, 3, 3)

In [25]:
mdp = QuickMDP(Evacuation,
    states       = 𝒮,
    actions      = 𝒜,
    transition   = T,
    reward       = R,
    discount     = γ,
    initialstate = 𝒮, # initialstate_array,
    isterminal   = termination,
    render       = render,
    statetype    = statetype #
    );

In [26]:
#render(mdp)

In [27]:
solver = ValueIterationSolver(max_iterations=30, belres=1e-6, verbose=true);

**Policy**

In [28]:
# Set discount factor to variable gamma and solve the MDP to obptain policy pi mapping states to actions a

In [29]:
@show policy = solve(solver, mdp) 

┌ Warning: Problem creating an ordered vector of states in ordered_states(...). There is likely a mistake in stateindex(...) or n_states(...).
│ 
│ n_states(...) was 102481.
│ 


│ states corresponding to the following indices were missing from states(...): [1, 2, 3, 4, 5, 6, 7, 8, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 81, 82, 83, 84, 85, 86, 87, 88, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 161, 162, 163, 164, 165, 166, 167, 168, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 241, 242, 243, 244, 245, 246, 247, 248, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 321, 322, 323, 324, 325,

5, 48346, 48347, 48348, 48349, 48350, 48351, 48352, 48353, 48354, 48355, 48356, 48357, 48358, 48359, 48360, 48361, 48362, 48363, 48364, 48365, 48366, 48367, 48368, 48369, 48370, 48371, 48372, 48373, 48374, 48375, 48376, 48377, 48378, 48379, 48380, 48381, 48382, 48383, 48384, 48401, 48402, 48403, 48404, 48405, 48406, 48407, 48408, 48425, 48426, 48427, 48428, 48429, 48430, 48431, 48432, 48433, 48434, 48435, 48436, 48437, 48438, 48439, 48440, 48441, 48442, 48443, 48444, 48445, 48446, 48447, 48448, 48449, 48450, 48451, 48452, 48453, 48454, 48455, 48456, 48457, 48458, 48459, 48460, 48461, 48462, 48463, 48464, 48481, 48482, 48483, 48484, 48485, 48486, 48487, 48488, 48505, 48506, 48507, 48508, 48509, 48510, 48511, 48512, 48513, 48514, 48515, 48516, 48517, 48518, 48519, 48520, 48521, 48522, 48523, 48524, 48525, 48526, 48527, 48528, 48529, 48530, 48531, 48532, 48533, 48534, 48535, 48536, 48537, 48538, 48539, 48540, 48541, 48542, 48543, 48544, 48561, 48562, 48563, 48564, 48565, 48566, 48567, 485

 63948, 63949, 63950, 63951, 63952, 63953, 63954, 63955, 63956, 63957, 63958, 63959, 63960, 63961, 63962, 63963, 63964, 63965, 63966, 63967, 63968, 63969, 63970, 63971, 63972, 63973, 63974, 63975, 63976, 63977, 63978, 63979, 63980, 63981, 63982, 63983, 63984, 64001, 64002, 64003, 64004, 64005, 64006, 64007, 64008, 64025, 64026, 64027, 64028, 64029, 64030, 64031, 64032, 64033, 64034, 64035, 64036, 64037, 64038, 64039, 64040, 64041, 64042, 64043, 64044, 64045, 64046, 64047, 64048, 64049, 64050, 64051, 64052, 64053, 64054, 64055, 64056, 64057, 64058, 64059, 64060, 64061, 64062, 64063, 64064, 64081, 64082, 64083, 64084, 64085, 64086, 64087, 64088, 64105, 64106, 64107, 64108, 64109, 64110, 64111, 64112, 64113, 64114, 64115, 64116, 64117, 64118, 64119, 64120, 64121, 64122, 64123, 64124, 64125, 64126, 64127, 64128, 64129, 64130, 64131, 64132, 64133, 64134, 64135, 64136, 64137, 64138, 64139, 64140, 64141, 64142, 64143, 64144, 64161, 64162, 64163, 64164, 64165, 64166, 64167, 64168, 64185, 64186

9550, 79551, 79552, 79553, 79554, 79555, 79556, 79557, 79558, 79559, 79560, 79561, 79562, 79563, 79564, 79565, 79566, 79567, 79568, 79569, 79570, 79571, 79572, 79573, 79574, 79575, 79576, 79577, 79578, 79579, 79580, 79581, 79582, 79583, 79584, 79601, 79602, 79603, 79604, 79605, 79606, 79607, 79608, 79625, 79626, 79627, 79628, 79629, 79630, 79631, 79632, 79633, 79634, 79635, 79636, 79637, 79638, 79639, 79640, 79641, 79642, 79643, 79644, 79645, 79646, 79647, 79648, 79649, 79650, 79651, 79652, 79653, 79654, 79655, 79656, 79657, 79658, 79659, 79660, 79661, 79662, 79663, 79664, 79681, 79682, 79683, 79684, 79685, 79686, 79687, 79688, 79705, 79706, 79707, 79708, 79709, 79710, 79711, 79712, 79713, 79714, 79715, 79716, 79717, 79718, 79719, 79720, 79721, 79722, 79723, 79724, 79725, 79726, 79727, 79728, 79729, 79730, 79731, 79732, 79733, 79734, 79735, 79736, 79737, 79738, 79739, 79740, 79741, 79742, 79743, 79744, 79761, 79762, 79763, 79764, 79765, 79766, 79767, 79768, 79785, 79786, 79787, 79788, 

52, 95153, 95154, 95155, 95156, 95157, 95158, 95159, 95160, 95161, 95162, 95163, 95164, 95165, 95166, 95167, 95168, 95169, 95170, 95171, 95172, 95173, 95174, 95175, 95176, 95177, 95178, 95179, 95180, 95181, 95182, 95183, 95184, 95201, 95202, 95203, 95204, 95205, 95206, 95207, 95208, 95225, 95226, 95227, 95228, 95229, 95230, 95231, 95232, 95233, 95234, 95235, 95236, 95237, 95238, 95239, 95240, 95241, 95242, 95243, 95244, 95245, 95246, 95247, 95248, 95249, 95250, 95251, 95252, 95253, 95254, 95255, 95256, 95257, 95258, 95259, 95260, 95261, 95262, 95263, 95264, 95281, 95282, 95283, 95284, 95285, 95286, 95287, 95288, 95305, 95306, 95307, 95308, 95309, 95310, 95311, 95312, 95313, 95314, 95315, 95316, 95317, 95318, 95319, 95320, 95321, 95322, 95323, 95324, 95325, 95326, 95327, 95328, 95329, 95330, 95331, 95332, 95333, 95334, 95335, 95336, 95337, 95338, 95339, 95340, 95341, 95342, 95343, 95344, 95361, 95362, 95363, 95364, 95365, 95366, 95367, 95368, 95385, 95386, 95387, 95388, 95389, 95390, 95

LoadError: KeyError: key State(6227936160, 6227937167, 5, -2) not found

In [30]:
@requirements_info ValueIterationSolver() mdp()

┌ Warning: POMDPs.@requirements_info is deprecated, use POMDPLinter.@requirements_info instead.
└ @ POMDPs /Users/lisaeinstein/.julia/packages/POMDPs/PFpk1/src/deprecated.jl:28


LoadError: MethodError: objects of type QuickMDP{Evacuation, State, Action, NamedTuple{(:stateindex, :isterminal, :states, :statetype, :discount, :render, :actions, :actionindex, :transition, :reward, :initialstate), Tuple{Dict{State, Int64}, typeof(termination), Vector{Any}, DataType, Float64, typeof(render), Vector{Action}, Dict{Action, Int64}, typeof(T), typeof(R), Vector{Any}}}} are not callable

In [31]:
# a = action(policy, s) # returns the optimal action for state s
# value(policy, s) # returns the optimal value at state s


**Additional possible needs**

In [ ]:
# one based policy? accept averyone? discuss this chunk of code 


# begin
# 	function one_based_policy!(policy)
# 		# change the default action in the policy (all zeros) to all ones (if needed)
# 		if all(iszero, policy.policy)
# 			policy.policy[:] = ones(eltype(policy.policy), length(policy.policy))
# 		end
# 	end
#     function get_rewards(mdp::QuickMDP{GridWorld}, policy::Policy)
#         null_state = params.null_state
#         valid_states = setdiff(states(mdp), [null_state])
#         U = map(s->reward(mdp, s), valid_states)
#     end
#     function values(mdp::QuickMDP{GridWorld}, policy::Policy)
#         null_state = params.null_state
#         valid_states = setdiff(states(mdp), [null_state])
#         U = map(s->value(policy, s), valid_states)
#     end
#     function values(mdp::QuickMDP{GridWorld}, planner::MCTSPlanner)
#         null_state = params.null_state
#         valid_states = setdiff(states(mdp), [null_state])
#         U = []
#         for s in valid_states
#             u = 0
#             try
#                 u = value(planner, s)
#             catch
#                 # state not in tree
#             end
#             push!(U, u)
#         end
#         return U
#     end
#     function values(mdp::QuickMDP{GridWorld}, policy::ValuePolicy)
#         maxU = mapslices(maximum, policy.value_table, dims=2)
#         return maxU[1:end-1] # remove null_state
#     end
#     struct NothingPolicy <: Policy end
    
#     # Use this to get a stationary grid of rewards
#     function values(mdp::QuickMDP{GridWorld}, policy::Union{NothingPolicy, FunctionPolicy})
#         null_state = params.null_state
#         valid_states = setdiff(states(mdp), [null_state])
#         rewards = map(s->reward(mdp, s), valid_states)
#     end
# end

**Visualizations**

In [34]:
struct evacPolicy <: Policy end 

In [38]:
function plot_evac(mdp::MDP,
        policy::Policy=evacPolicy(),  # our policy will have states State(c, t, f, v) with actions (reject/accept)
        iter=0,
        discount=NaN; 
        outline=true,
        show_policy=true,
        show_rewards=false, 
        outline_state::Union{State, Nothing}=nothing
    )
                   
    gr()
    
    (xmax, ymax) = params.size # reshape grid to family size * visa status
    Uxy = reshape(U, xmax, ymax)
    # plot values (i.e the U matrix)
    fig = heatmap(Uxy',
                  legend=:none,
                  aspect_ratio=:equal,
                  framestyle=:box,
                  tickdirection=:out)
    xlims!(0.5, xmax+0.5)
    ylims!(0.5, ymax+0.5)
    xticks!(1:xmax)
    yticks!(1:ymax)
    rectangle(w, h, x, y) = Shape(x .+ [0,w,w,0], y .+ [0,0,h,h])
    # not sure the visualization makes perfect sense since x and y will not have every possible family size...
    # policy_grid(policy::Policy, xmax::Int, ymax::Int, time::Int, capacity::Int)
    # Let x represent family size, y represent visa status 
    for x in 1:xmax, y in 1:ymax 
        policy_grid(policy, xmax, ymax, 0, 0) # time = 0 and capacity = 0 here - the last time step 
        annotate!([(x, y, (grid[x,y], :center, 12, "Computer Modern"))])
        rect = rectangle(1, 1, x - 0.5, y - 0.5)
        plot!(rect, fillalpha=0, linecolor=:gray)
        
        # Some if statement here to say: 
        # if policy at State(c,t,f,v) == ACCEPT
            # color = green
        # else 
            # no color 
    end
        title!("Evacuation Policy Plot$extra_title")
        return fig 
end

plot_evac (generic function with 4 methods)

In [ ]:
policy_grid